# Algorithm Testing
A notebook used to compare algorithm performance.


In [1]:
### Imports
from nitools import utils 
from nitools.classifiers import PAE_RVFL, ELM, AE_ELM, PAE_ELM
from nitools.models import LeNetPlus, LRF_ELM, LRF_ELMplus, MiniRes
from nitools.operations import resetseed

import numpy as np
import random
import torch
import time as t

In [2]:
# Import dataset
mnist = utils.load_mnist(augment=True)

mnist_in = 784
mnist_class = 10

In [3]:
def run(model, n=10000, nt=10000, data=mnist):
    X = None
    y = None 
    tX = None
    try:
        X = data['train_X'].float()[:n]
        y = data['train_y'].float()[:n]
        tX = data['test_X'].float()[:nt]
    except:
        X = data['train_X'].view((50000,3,32,32)).float()[:n]
        y = torch.from_numpy(data['train_y']).float()[:n]
        tX = data['test_X'].view((10000,3,32,32)).float()

    train_start = t.time()
    result = model.train(X, y)
    train_end = t.time()
    

    test_start = t.time()
    pred = model.predict(tX)
    test_end = t.time()

    pred_arg = np.zeros(nt)

    for i in range(len(pred)):
        pred_arg[i] = np.argmax(pred[i])


    print(f"train: {train_end-train_start}")
    print(f"test: {test_end-test_start}")
    utils.evaluation_summary('lenet  - MNIST', pred_arg, data['test_y'][:nt])
    
def runos(model, b=5000, iters=1, nt=10000, data=mnist):
    X = torch.from_numpy(data['train_X']).view((60000,1,28,28)).float()
    y = torch.from_numpy(data['train_y']).float()

    train_start = t.time()
    for i in range(0,60000,b):
        if i/b==iters:
            break
        result = model.train(X[i:i+b], y[i:i+b])
    train_end = t.time()
    
    tX = torch.from_numpy(data['test_X']).view((10000,1,28,28)).float()

    test_start = t.time()
    pred = model.predict(tX)[:nt]
    test_end = t.time()

    pred_arg = np.zeros(nt)

    for i in range(len(pred)):
        pred_arg[i] = np.argmax(pred[i])


    print(f"train: {train_end-train_start}")
    print(f"test: {test_end-test_start}")
    utils.evaluation_summary('lenet  - MNIST', pred_arg, data['test_y'])

## 1. MNIST

Dataset of greyscale 28x28 images of handwritten digits. 

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*


In [ ]:
data = utils.load_mnist(augment=False, label_smoothing=0.1)
SEED = 2359487
resetseed(SEED)

model = LRF_ELM()

run(model, n=5000, nt=10000, data=data)

torch.Size([5000, 30000])


In [ ]:
data = utils.load_mnist(augment=False)
SEED = 2359487
resetseed(SEED)

model = LRF_ELM(c=1000, _lambda=1, p=0)

run(model, n=25000, nt=10000, data=data)

## 2. CIFAR-10

Dataset of colour (3-dimension) 32x32 images of objects of 10 classes:

0.	airplane
1.	automobile
2.	bird
3.	cat
4.	deer
5.	dog
6.	frog
7.	horse
8.	ship
9.	truck

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*

In [4]:
# Import dataset
cifar10 = utils.load_cifar10(scaled=True, augment=False)

cifar_in = 32*32*3
cifar_class = 10

In [6]:
SEED = 22
resetseed(SEED)

model = LeNetPlus(in_channels=3, c=1000, h_size=900)

run(model, n=40000, nt=5000, data=cifar10)

torch.Size([40000, 1152])
train: 218.8179907798767
test: 1.8466460704803467
Evaluation for: lenet  - MNIST
Classifier 'lenet  - MNIST' has Acc=0.370 P=0.369 R=0.365 F1=0.363
              precision    recall  f1-score   support

         0.0      0.357     0.426     0.388       408
         1.0      0.465     0.388     0.423       606
         2.0      0.305     0.347     0.324       450
         3.0      0.187     0.289     0.227       322
         4.0      0.318     0.372     0.343       433
         5.0      0.225     0.308     0.260       357
         6.0      0.446     0.372     0.406       588
         7.0      0.406     0.338     0.369       595
         8.0      0.522     0.417     0.463       631
         9.0      0.464     0.390     0.424       610

    accuracy                          0.370      5000
   macro avg      0.369     0.365     0.363      5000
weighted avg      0.392     0.370     0.377      5000


Confusion matrix:
 [[174  32  51  10  21  13  36  33  85  33]
 [ 2